## **Import library**

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
from scipy.sparse import csr_matrix

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## **Import file**

In [ ]:
movie_names = pd.read_csv("movies_metadata.csv")
movie_names.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
ratings_data = pd.read_csv("ratings_small.csv")
ratings_data = ratings_data.drop('timestamp', axis = 1)
ratings_data

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


## **Preprocessing**

In [ ]:
movie_data = pd.concat([ratings_data, movie_names], axis=1)
movie_data = movie_data.drop(['adult', 'belongs_to_collection', 'budget', 'genres',
                              'homepage', 'id', 'imdb_id', 'original_language',
                              'original_title', 'overview', 'popularity', 'poster_path',
                              'production_companies', 'production_countries',
                              'release_date', 'revenue', 'runtime', 'spoken_languages',
                              'status', 'tagline', 'video', 'vote_average', 'vote_count' ], axis=1)
movie_data.head()

,userId,movieId,rating,title
0,1,31,2.5,Toy Story
1,1,1029,3.0,Jumanji
2,1,1061,3.0,Grumpier Old Men
3,1,1129,2.0,Waiting to Exhale
4,1,1172,4.0,Father of the Bride Part II


In [ ]:
print(movie_data['rating'].isnull().sum())
print(movie_data['movieId'].isnull().sum())
print(movie_data['userId'].isnull().sum())
print(movie_data['title'].isnull().sum())

0
0
0
54544


In [ ]:
movie_data.duplicated().sum()

0

In [ ]:
movie_data = movie_data[movie_data['title'].notna()]
movie_data.head()

,userId,movieId,rating,title
0,1,31,2.5,Toy Story
1,1,1029,3.0,Jumanji
2,1,1061,3.0,Grumpier Old Men
3,1,1129,2.0,Waiting to Exhale
4,1,1172,4.0,Father of the Bride Part II


## **trend**

In [ ]:
grouped_data = movie_data.groupby('title')
trend_list = []
for name, group in grouped_data:
  trend_list.append( list([name,group['rating'].mean()]))
trend = pd.DataFrame(trend_list, columns=('title', 'rating')).sort_values('rating',ascending=False)
print(trend)


                               title  rating
25324                           Rapt     5.0
21730     My Sister's Kids in Africa     5.0
21907      Nasu: Summer in Andalusia     5.0
5317               Borderline Normal     5.0
21940                    Native Land     5.0
...                              ...     ...
39171                        Tremors     0.5
5981                  Cabaret Balkan     0.5
34640  The Lost World: Jurassic Park     0.5
34632                   The Lost Son     0.5
39535      Two Weeks in Another Town     0.5

[42277 rows x 2 columns]


## **Pivot**

In [ ]:
movies_users = movie_data.pivot(index=['movieId'], columns=['userId'], values='rating').fillna(0)
movies_users.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,0.0,3.0,3.5,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,4.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,5.0,4.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,4.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,5.0,4.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movies_users[(movies_users<=3) & (movies_users!=0)] = -1
movies_users[movies_users>3] = 1
movies_users

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324
movieId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161594,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
mat_movies_users=csr_matrix(movies_users.values)
mat_movies_users

<6477x324 sparse matrix of type '<class 'numpy.float64'>'
	with 45460 stored elements in Compressed Sparse Row format>

## **recommend**

In [ ]:
model_knn= NearestNeighbors(metric='euclidean', algorithm='auto', n_neighbors=20, n_jobs=-1).fit(mat_movies_users)
model_knn

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [ ]:
def ConverMovieIdTOIndex(movieId):
  movies_id = movie_data['movieId']
  movies_id = movies_id.drop_duplicates().sort_values()
  movies_id = movies_id.reset_index(drop=True)
  dict_movie_id = movies_id.to_dict()
  keys = list(dict_movie_id.keys())
  vals = list(dict_movie_id.values())
  if movieId in vals:
    return(keys[vals.index(movieId)])
  else:
    return(-1)


In [ ]:
def RecommenderBasedOnTheMovie(movie_name, data, n_recommendations):
  movie_index = process.extractOne(movie_name, movie_names['title'])[2]
  print('Movie Selected: ',movie_names['title'][movie_index], ', Index: ', movie_index)
  print('Searching for recommendations.....')
  recommend_frame = []
  movie_id = ConverMovieIdTOIndex(movie_index)
  if(movie_id != -1):
    distances, indices = model_knn.kneighbors(data[movie_id], n_neighbors=n_recommendations)
    indices = indices.squeeze()
    distances = distances.squeeze()
    for i in range(len(indices) - 1):
      if indices[i] == movie_index:
        indices = np.delete(indices, i)
        distances = np.delete(distances,i)
    recc_movie_indices = sorted(list(zip(indices,distances)),key=lambda x: x[1])[0:-1:]
    for val in recc_movie_indices:
        recommend_frame.append([movie_names['title'][val[0]],val[1]])     
  return recommend_frame

In [ ]:
def recommendedTrendMovies(n_recommendation):
  df= pd.DataFrame(trend.head(n_recommendation), columns=('title', 'rating'))

  return df;

In [ ]:
def RecommenderBasedOnTheUser(userId, data, n_recommendation, my_movies):
  recommended_movies = []
  for i in my_movies['title']:
    recommended_movies.append(RecommenderBasedOnTheMovie(i, data, math.ceil(n_recommendation / len(my_movies))+ 2))
  recommended_movies = np.unique(recommended_movies, axis = 0)
  if np.all(recommended_movies== 0):
    np.delete(recommended_movies, [])
    df=pd.DataFrame()
  else:
    recommended_movies = np.resize(np.array(recommended_movies), (len(my_movies)*(n_recommendation - 1), 2))
    recommended_movies = np.unique(recommended_movies, axis = 0)
    recommended_movies = sorted(recommended_movies,key=lambda x: x[1])
    if recommended_movies == []:
      np.delete(recommended_movies, [])
      np.delete(recommended_movies,[])
    df = pd.DataFrame(recommended_movies[:n_recommendations], columns=('title', 'Distance'))
  
  return df;

In [ ]:
def movieRecommender(userId, data, n_recommendation):
  my_movies = movie_data[movie_data['userId'] == userId]
  my_movies = my_movies[my_movies['rating'] >= 4]
  if len(my_movies) == 0:
    df = recommendedTrendMovies(n_recommendation)
  else:
    if len(my_movies)> n_recommendations:
        df = RecommenderBasedOnTheUser(userId, data, n_recommendation, my_movies[0: n_recommendations] )
    else:
      df = RecommenderBasedOnTheUser(userId, data, n_recommendation, my_movies )
    if len(df)<n_recommendation:
      df = df.append(recommendedTrendMovies(n_recommendation-len(df)))
  df_title = pd.DataFrame(df.iloc[:,0])
  df_title = df_title.reset_index(drop=True)
  
  return df_title;

In [ ]:
n_recommendations = 2
df = movieRecommender(3, mat_movies_users, n_recommendations)
df

Movie Selected:  Heidi Fleiss: Hollywood Madam , Index:  97
Searching for recommendations.....
Movie Selected:  Mr. Wrong , Index:  100
Searching for recommendations.....


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


,title
0,Last Summer in the Hamptons
1,Ri¢hie Ri¢h
